In [22]:
import requests
import base64
from typing import Tuple

YOUR_EMAIL = "alvarmaciel@gmail.com"

# URL base del desafío
BASE_URL = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/{YOUR_EMAIL}/register"

print(f"URL configurada: {BASE_URL}")

URL configurada: https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/alvarmaciel@gmail.com/register


In [23]:
def register_user(email: str, data: str) -> bytes:
    """
    Registra un usuario y obtiene el perfil cifrado.
    
    Args:
        email: Dirección de correo
        data: Datos adicionales (sin ; ni =)
    
    Returns:
        Bytes del IV + ciphertext
    """
    # Codificar data en base64
    data_b64 = base64.b64encode(data.encode()).decode()
    
    # Hacer POST request
    response = requests.post(
        BASE_URL,
        files={
            'email': email,
            'data': data_b64
        }
    )
    
    if response.status_code != 200:
        raise Exception(f"Error en la petición: {response.status_code}\n{response.text}")
    
    # Decodificar respuesta base64
    encrypted_profile = base64.b64decode(response.text.strip())
    
    return encrypted_profile

def split_iv_ciphertext(encrypted: bytes) -> Tuple[bytes, bytes]:
    """
    Separa el IV (primeros 16 bytes) del ciphertext.
    """
    iv = encrypted[:16]
    ciphertext = encrypted[16:]
    return iv, ciphertext

def visualize_blocks(data: bytes, block_size: int = 16) -> None:
    """
    Visualiza los bloques de datos.
    """
    for i in range(0, len(data), block_size):
        block = data[i:i+block_size]
        print(f"Bloque {i//block_size}: {block.hex()} ({len(block)} bytes)")

print("Funciones auxiliares definidas.")

Funciones auxiliares definidas.


In [54]:
# Probemos con un email y datos simples para entender la estructura
test_email = "test@example.com"
test_data = "TestData"

print(f"Registrando usuario: {test_email}")
print(f"Con datos: {test_data}")
print()

# El perfil resultante será:
expected_profile = f"user={test_email};data={test_data};role=user"
print(f"Perfil esperado (plaintext): {expected_profile}")

print("\n--- Bloques a encriptar ---")

for i in range(0, len(expected_profile),16):
    
    block=expected_profile[i:i+16]
    print(f"Bloque {i//16}: {block} ({len(block)} bytes)")
    
print(f"Longitud: {len(expected_profile)} bytes")
print()

# Obtener el perfil cifrado
encrypted = register_user(test_email, test_data)
print(f"Perfil cifrado recibido: {len(encrypted)} bytes")
print(f"Perfil: {encrypted}")

# Separar IV y ciphertext
iv, ciphertext = split_iv_ciphertext(encrypted)
print(f"\nIV ({len(iv)} bytes): {iv.hex()}")
print(f"Ciphertext ({len(ciphertext)} bytes):")
visualize_blocks(ciphertext)

Registrando usuario: test@example.com
Con datos: TestData

Perfil esperado (plaintext): user=test@example.com;data=TestData;role=user

--- Bloques a encriptar ---
Bloque 0: user=test@exampl (16 bytes)
Bloque 1: e.com;data=TestD (16 bytes)
Bloque 2: ata;role=user (13 bytes)
Longitud: 45 bytes

Perfil cifrado recibido: 64 bytes
Perfil: b'T\x1c\xeaQG)\x9dq\x97\xaa\x0e0\xd5\xe8\xe1\x8bv\xd4\xbd\x15L\x970\xba\x0b\xb0\xee\xec\xf5\x1c\xf1\x03\xd4|?\xe5H~mnt\xae\xc0\x94\xe2\xb0B\x95\xc7Yi\x05\xcf\x13,_\xc2\x13\t\x93>F\x91<'

IV (16 bytes): 541cea5147299d7197aa0e30d5e8e18b
Ciphertext (48 bytes):
Bloque 0: 76d4bd154c9730ba0bb0eeecf51cf103 (16 bytes)
Bloque 1: d47c3fe5487e6d6e74aec094e2b04295 (16 bytes)
Bloque 2: c7596905cf132c5fc21309933e46913c (16 bytes)


In [55]:
# Calculemos las posiciones
# Formato: user=<email>;data=<datos>;role=user

def calculate_positions(email: str, data: str) -> dict:
    """
    Calcula las posiciones de cada parte del perfil.
    """
    profile = f"user={email};data={data};role=user"
    
    role_start = profile.index("role=")
    user_start = profile.index("user", role_start)  # El 'user' después de 'role='
    
    return {
        'profile': profile,
        'length': len(profile),
        'role_start': role_start,
        'user_start': user_start,
        'user_block': user_start // 16,
        'user_offset': user_start % 16 # Posición relativa donde comienza user
    }

# Ejemplo
info = calculate_positions(test_email, test_data)
print("Información del perfil:")
for key, value in info.items():
    print(f"  {key}: {value}")

# Visualizar bloques del plaintext
print("\nBloques del plaintext (esperado):")
plaintext_bytes = info['profile'].encode()
visualize_blocks(plaintext_bytes)

Información del perfil:
  profile: user=test@example.com;data=TestData;role=user
  length: 45
  role_start: 36
  user_start: 41
  user_block: 2
  user_offset: 9

Bloques del plaintext (esperado):
Bloque 0: 757365723d74657374406578616d706c (16 bytes)
Bloque 1: 652e636f6d3b646174613d5465737444 (16 bytes)
Bloque 2: 6174613b726f6c653d75736572 (13 bytes)


In [56]:
original = b'role=user'
target = b'role=admin'
mask = bytes(a ^ b for a, b in zip(original, target))
mask

b'\x00\x00\x00\x00\x00\x14\x17\x08\x1b'

In [64]:
modified_blocks = bytearray(ciphertext)
role_start = info["role_start"]
role_block_index = info["user_block"]

# Necesitas modificar 9 bytes del bloque anterior
start_pos = role_start
end_pos = role_start + len(mask)
modified_blocks[start_pos:end_pos] = bytes(
    a ^ b for a, b in zip(modified_blocks[start_pos:end_pos], mask)
)
modifyed_cypher = iv + bytes(modified_blocks)
modified_encypted = base64.b64encode(modified_cypher).decode()

In [65]:
challenge_response = requests.post(
    f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/{YOUR_EMAIL}/answer",
    files = { "message":modifyed_encrypted}
)
print(f"Status code: {challenge_response.status_code}")
print(challenge_response.text)

Status code: 200
Lo siento, siga participando.

